# Global indicator project
## Get and save study region walkable street network, load OSM basic network and stats

This notebook describes functions to get street networks from OpenStreetMap (OSM) using OSMnx, a Python library that lets you download spatial geometries and construct, project, and visualize street networks from OpenStreetMap's API. This notebook include functions to obtain OSM network and basic stats based on study region buffer polygon (created using at 01_create_studyregion). The network data are saved to local data folder.   



In [1]:
# Libraries used for OSMnx analyses and output
import networkx as nx
import time 
import osmnx as ox
import matplotlib.pyplot as plt
import numpy as np
import requests
import pandas as pd
import geopandas as gpd

from shapely.geometry import shape,Point, LineString, Polygon


ox.config(use_cache=True, log_console=True)

### Set up project config

In [2]:
placename = 'phoenix' # placename

region = 'Arizona, USA' # study region name

suffix = '_201905' # output data time

buffer_dist = 1e4 # study region buffer 10km

data_folder = '../data/Maricopa_County' # studyregion administrative data folder 

OSM_folder = '../data/OSM' # folder path to save OSM data 

shape_filename = 'MC_Boundary.shp' # study region boundary filename

filepath = data_folder + '/' + shape_filename # study region boundary filepath

## Get pedestrain street network and save to local folder

In [5]:
# Extract complete OSM network and save local graph and edge shapefile: "all (non-private) OSM streets and paths"
def Save_OSM_G_gdf(polygon, network_type= 'walk', placename=placename, suffix=suffix, folder=OSM_folder):    
    """
    save a graphml and GeoDataFrame from a single polygon.
    Parameters
    ----------
    polygon : shapely Polygon or MultiPolygon
        the shape to get network data within. coordinates should be in units of
        latitude-longitude degrees.
    network_type : string
        what type of street network to get. Default type is pedestrain network
    placename: string
        place name
    suffix: string
        output data date
    folder : string
        where to save the shapefile, specify local folder path for OSM resource
    Returns
    -------
    none

    """
    #The graph_from_polygon function requires a polygon in units of lat-long (epsg 4326)
    G = ox.graph_from_polygon(polygon, network_type=network_type, retain_all = True)
    #save network as graphml
    ox.save_graphml(G, filename='{studyregion}_{network_type}{suffix}.graphml'.format(
        studyregion = placename, network_type=network_type, suffix = suffix), folder=folder)
    
    #set up project projection, and save the projected graph (project to UTM so we can use metres as a unit when buffering)
    G_proj = ox.project_graph(G)
    ox.save_graphml(G_proj, filename='{studyregion}_proj_{network_type}{suffix}.graphml'.format(
        studyregion = placename, network_type=network_type, suffix = suffix), folder=folder)
    
    #save projected network geodataframe as shapefile (project to UTM so we can use metres as a unit when buffering)
    edges_proj_gdfs = ox.graph_to_gdfs(G_proj, nodes=False, edges=True, fill_edge_geometry=True)
    ox.save_gdf_shapefile(edges_proj_gdfs, filename='{studyregion}_proj_{network_type}{suffix}'.format(
        studyregion = placename, network_type=network_type, suffix = suffix), folder=folder)
    
    #show network figure
    fig, ax = plt.subplots(figsize=(5, 5))
    ax = edges_proj_gdfs.plot(ax=ax)
    ax.set_title(address)
    ax.set_axis_off()
    fig.suptitle('{} OSM {} street network'.format(placename, network_type), fontsize=14, fontweight='bold')
    plt.show()


## Load pedestrain street network from local folder

In [3]:
def get_OSM_G(G_filename, OSM_folder, polygon=None, network_type= 'walk'):
    """
    If studyregion graphml file exist:
    Load a GraphML file from disk
    
    Else:
    Query from OSM from studyregion polygon to get the network

    Parameters
    ----------
    G_filename : string
        the name of the graphml file (including file extension)
    OSM_folder : string
        the folder containing the OSM file, if None, use default data folder
    polygon : shapely Polygon or MultiPolygon
        the shape to get network data within. coordinates should be in units of
        latitude-longitude degrees.
    network_type : string
        what type of street network to get. Default type is pedestrain network  

    Returns
    -------
    networkx multidigraph
    """
    if os.path.exists(OSM_folder +'/' + G_filename):
        G = ox.load_graphml(filename=G_filename, folder=OSM_folder)
    else:
        G = ox.graph_from_polygon(polygon, network_type=network_type, retain_all = True)
    return G
        
    

In [4]:
def get_OSM_edges_gdf(shapefile_path, OSM_folder, polygon, network_type= 'walk'):
    """
    If studyregion gdf file exist:
    Load a (projected) edge shapefile from disk 
    
    Else: query OSM to get the network gdf.

    Parameters
    ----------
    shapefile_path : string
        the name of the shapefile path(including file extension)
    OSM_folder : string
        the folder containing the OSM file, if None, use default data folder
    polygon : shapely Polygon or MultiPolygon
        the shape to get network data within. coordinates should be in units of
        latitude-longitude degrees.
    network_type : string
        what type of street network to get. Default type is pedestrain network  

    Returns
    -------
    GeoDataFrame
    """
    if os.path.exists(shapefile_path):
        edges = gpd.GeoDataFrame.from_file(shapefile_path)
    else:
        G = ox.graph_from_polygon(polygon, network_type=network_type, retain_all = True)
        G_proj = ox.project_graph(G)
        edges = ox.graph_to_gdfs(G_proj, nodes=False, edges=True, fill_edge_geometry=True)
    return edges

### Load OSM network stats
This function retains all the basic stats for all streets, pedestrain network, and cycle network within 10km buffered study regions. The OSM street network data are loaded from local data folder. 

In [8]:
def load_OSM_stats(G_filename, folder=OSM_folder):
    """
    retains all the basic stats for all, pedestrain network, and 
    cycle network within study regions graphml from a local folder
    
    Parameters
    ----------
    G_filename : string
        the name of the graphml file (including file extension)
    OSM_folder : string
        the folder containing the OSM file, if None, use default data folder
    
    
    Returns
    -------
    DataFrame
    """
    df = pd.DataFrame()
    #load street network data from local directory
    for networktype in ['all', 'walk', 'bike']:
        G = ox.load_graphml(G_filename, folder=folder)
        
        gdf_nodes_proj = ox.graph_to_gdfs(G_proj, edges=False)
        graph_area_m = gdf_nodes_proj.unary_union.convex_hull.area
        
        stats = ox.basic_stats(G_proj, area=graph_area_m, clean_intersects=True, circuity_dist='euclidean', tolerance=15)
        df1 = pd.DataFrame.from_dict(stats, orient='index', columns=[networktype + '_' + place])
        df = pd.concat([df, df1], axis=1)
    return df
    